In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.m2.ssro import ssro, sequence
import scipy.fftpack
import win32com.client #imports the pywin32 library
%matplotlib inline

In [ ]:
def arg_value(t, t_0):
    ind = np.argmin(np.abs(t - t_0))
    return ind  

def fourier_complex(signal, time):
    """Returns the Fouier transform of a single signal with time, including the negative frequencies."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    return np.fft.fftshift(fft), np.fft.fftshift(freq)

def fourier(signal, time):
    """Returns the Fouier transform of a single signal with time, including the negative frequencies."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    return np.abs(np.fft.fftshift(fft)), np.fft.fftshift(freq)

def fourier_pos_freq(signal, time):
    """Returns the Fouier transform of a single signal with time."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    n = len(fft)
    fft = fft[0:n//2]
    freq = freq[0:n//2]
    return np.abs(fft), freq

def hermite_pulse_env(env_amplitude, T_herm, mu=0):
    t = np.linspace(-25e-7,25e-7,int(1e5)) + mu
    env = env_amplitude*(1-0.956*((t-mu)/T_herm)**2)*np.exp(-((t-mu)/T_herm)**2)
    return env, t

def hermite_pulse(env_amplitude, T_herm, frequency, mu=0):
    env, t = hermite_pulse_env(env_amplitude, T_herm, mu=0)
    hermite_pulse = env * np.sin(t*2*np.pi*frequency)
    return hermite_pulse, t + mu

def path_folder(disk='D', when='latest', multiple_msmt=False, older_than=None, newer_than=None):
    """Returns a list with string with the path to the measurements."""
    if disk == 'D':
        data_folder = None
    else:
        data_folder = '{}:/data'.format(disk)
    if multiple_msmt == False:
        if when == 'latest':
            msmt_folder = tb.latest_data('mw_pulse_msmt', folder=data_folder)
        else:
            msmt_folder = tb.data_from_time(when, folder=data_folder)
    else:
        msmt_folder = tb.latest_data('mw_pulse_msmt', folder='{}:/data'.format(disk),
                                     newer_than=newer_than, older_than=older_than, return_all=True)
    return msmt_folder

class mw_msmt:
    def __init__(self, setup, where, disk='D', when='latest', old_measurement=False, incl_awg_signal = False):
        msmt_folder = path_folder(disk, when)
        f = sequence.SequenceAnalysis(msmt_folder, hdf5_mode='r')
        waveform = np.array(f.g['oscilloscope_msmt']['data'])
        self.length = f.g.attrs['Hermite_pi_length']
        self.power = f.g.attrs['mw_power']
        self.t_step = (waveform[0][-1] - waveform[0][0])/waveform.size
        self.number_of_pulses = int(f.g.attrs['multiplicity'])
        self.points_per_pulse = int(1e6 // self.number_of_pulses)
        if old_measurement:
            extra_points_per_pulse = int(round((self.length-100e-9)/self.t_step)/2)
            self.points_per_pulse += extra_points_per_pulse
            self.number_of_pulses -= 1 # Discard last pulse because it is not fully measured
        self.frequency = f.g.attrs['mw_frq'] + 30e6       
        self.t_herm = 0.1667*self.length
        self.t = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.amplitude = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.middle = np.zeros(self.number_of_pulses)
        self.offset = np.zeros(self.number_of_pulses)
        if incl_awg_signal:
            self.imod = np.array(f.g['pulses']['MW_Imod'])
            self.awg_time = np.array(f.g['pulses']['time'])
        if disk=='D':
            self.timestamp = msmt_folder[18:33]
        else:
            self.timestamp = msmt_folder[8:23]
        for pulse in range(self.number_of_pulses):
            self.t[pulse] = waveform[0][pulse*self.points_per_pulse:(pulse+1)*self.points_per_pulse]
            self.amplitude[pulse] = waveform[1][pulse*self.points_per_pulse:(pulse+1)*self.points_per_pulse]
            self.middle[pulse] = self.t[pulse][np.argmax(self.amplitude[pulse])]
#             self.offset[pulse] = self.amplitude[pulse][0:arg_value(self.t[pulse], self.t[pulse][np.argmax(self.amplitude)]-.75*self.length)].mean()
        self.offset = np.mean(self.amplitude[:][0:1000])
        self.amplitude -= self.offset # Corrects for a DC ofset of the signal
        self.t_step = (self.t[-1][-1] - self.t[0][0])/self.t.size
        self.setup = setup
        self.where = where
            
            
    @property
    def mean_fft(self):
        mean = np.mean(self.fft, axis=0)
        return mean
    
    @property
    def std_fft(self):
        std = np.std(self.fft, axis=0)
        return std
    

class pulse_msmt(mw_msmt):
    def __init__(self, setup, where, disk='D', when='latest', old_measurement=False, incl_awg_signal = False):
        mw_msmt.__init__(self, setup, where, disk, when, old_measurement, incl_awg_signal)
        self.fft = np.zeros((self.number_of_pulses, self.points_per_pulse//2))
        self.freq = np.zeros((self.number_of_pulses, self.points_per_pulse//2))
        for pulse in range(self.number_of_pulses):
            self.fft[pulse], self.freq[pulse] = fourier_pos_freq(self.amplitude[pulse], self.t[pulse])
        
    def integral_fft(self, lower_bound=None, upper_bound=None):
        integral = np.zeros(self.number_of_pulses)
        if lower_bound==None and upper_bound==None:
            for pulse in range(self.number_of_pulses):
                integral[pulse] = scipy.integrate.trapz(self.fft[pulse], self.freq[pulse])
        else:
            
            for pulse in range(self.number_of_pulses):
                ind_lower = arg_value(self.freq[pulse], lower_bound)
                ind_upper = arg_value(self.freq[pulse], upper_bound)
                integral[pulse] = scipy.integrate.trapz(self.fft[pulse][ind_lower:ind_upper], 
                                                        self.freq[pulse][ind_lower:ind_upper])
        return integral

class enveloppe_msmt(mw_msmt):
    def __init__(self, setup, where, disk='D', when='latest', old_measurement=False, incl_awg_signal = False):
        mw_msmt.__init__(self, setup, where, disk, when, old_measurement, incl_awg_signal)
        self.frequency = 0
        self.fft = np.zeros((self.number_of_pulses, self.points_per_pulse))
        self.freq = np.zeros((self.number_of_pulses, self.points_per_pulse))
        for pulse in range(self.number_of_pulses):
            self.fft[pulse], self.freq[pulse] = fourier(self.amplitude[pulse], self.t[pulse])

class sequential_pulse_msmts:
    def __init__(self, newer_than, older_than, disk='D'):
        msmt_folders = path_folder(disk, newer_than=newer_than, older_than=older_than, multiple_msmt=True)
        self.number_of_msmts = len(msmt_folders)
        self.msmts = []
        for msmt_folder in msmt_folders:
            when = tb.timestamp_from_datetime(tb.get_datetime_from_folder(msmt_folder))
            self.msmts.append(pulse_msmt(disk=disk, when=when))

In [ ]:
# a = enveloppe_msmt('X')
# a = enveloppe_msmt('X', when='20180323091723')
# a = enveloppe_msmt('Z', when='20180326125434')

# skew = pulse_msmt(setup='LT5', when='20180423102232', where='', disk='Z')
skew = pulse_msmt(setup='LT5', where='', disk='Z')

# power15 = pulse_msmt(setup='LT5', when='20180419150422', where='after the cryostat', disk='Z')
# power10 = pulse_msmt(setup='LT5', when='20180419150324', where='after the cryostat', disk='Z')
# power20 = pulse_msmt(setup='LT5', when='20180419150001', where='after the cryostat', disk='Z')
# lt_4_rf_source = pulse_msmt(setup='LT4', where='after the RF source', disk='Y', when='20180320144111', old_measurement=True)
# lt_4_amplifier = pulse_msmt(setup='LT4', where='after the amplifier', disk='Y', when='20180321111625', old_measurement=True)
# lt_4_switch = pulse_msmt(setup='LT4', where='after the RF switch', disk='Y', when='20180321120058', old_measurement=True)
# lt_4_cryo = pulse_msmt(setup='LT4', where='after the cryostat and sample', disk='Y', when='20180321134817')
# overnight = sequential_pulse_msmts(disk='Y',newer_than='20180321182000', older_than='20180322110000')

In [ ]:
# multi = overnight
msmt = skew
pulse = 0
measurement = 0

# Overview of the measurement

In [ ]:
plt.figure(figsize=(10,8))
for pulse in range(msmt.number_of_pulses):
    plt.plot(msmt.t[pulse], msmt.amplitude[pulse])
plt.xlabel('t(s)')
plt.ylabel('U(V)')
plt.title('Overview of the whole measurement, timestamp: {}'.format(msmt.timestamp))
plt.show()

# Analysis of single pulse

In [ ]:
# Plot waveform
plt.figure(figsize=(10,8))
# plt.plot(msmt.t[pulse], msmt.amplitude[pulse],label='Data')
# Plot ideal pulse
middle = (msmt.t[pulse][-1]-msmt.t[pulse][0])/2 + msmt.t[pulse][0]

if msmt.__class__.__name__ == 'enveloppe_msmt':
    env, t = hermite_pulse_env(env_amplitude=np.max(msmt.amplitude), mu=msmt.middle[pulse], T_herm=msmt.t_herm)
    plt.plot(t, env, '--', label='Ideal')
#     plt.plot(msmt.awg_time-1.9865e-5, msmt.imod/1.05, label='Send to AWG') #Plot input for the AWG
elif msmt.__class__.__name__ == 'pulse_msmt':
    env, t = hermite_pulse(env_amplitude=np.max(msmt.amplitude)*.96, frequency=msmt.frequency, mu=msmt.middle[pulse]+0e-8, T_herm=msmt.t_herm)
#     plt.plot(t, env, label='Ideal')
plt.plot(msmt.t[pulse], msmt.amplitude[pulse], label='Data')#, alpha=.7)
print "                       {} {}".format(msmt.setup, msmt.where)
    
time_window = 2e-7
# plt.xlim(msmt.middle[pulse]-4e-8, msmt.middle[pulse]-3e-8)
plt.xlim(msmt.middle[pulse]-time_window/2, msmt.middle[pulse]+time_window/2)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel('s')
plt.ylabel('V')
plt.title('Single Hermite pulse, timestamp : {}'.format(msmt.timestamp))
plt.legend()
plt.show()

# Plot the fourier transform
plt.figure(figsize=(8,5))
# plt.plot(msmt.freq[pulse], msmt.fft[pulse], label='Data')
# Plot ideal transform
fft, freq = fourier(env, t)
plt.plot(freq, fft, '--', label='Ideal')
plt.plot(msmt.freq[pulse], msmt.fft[pulse], label='Data')
    
plt.title('Fourier transform, timestamp : {}'.format(msmt.timestamp))
plt.xlabel('Frequency [Hz]')
plt.legend()
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.show()

plt.figure(figsize=(8,5))
fft_complex, freqe = fourier_complex(msmt.amplitude[pulse], msmt.t[pulse])
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.plot(freqe, np.unwrap(np.angle(fft_complex)), '--')

# Analysis of the pulses in one measurement

In [ ]:
half_width_integral = 30e6

plt.figure(figsize=(10,8))
for pulse in range(msmt.number_of_pulses):
    plt.plot(msmt.freq[pulse], msmt.fft[pulse], label='{}'.format(pulse))
plt.axvspan(msmt.frequency - half_width_integral, msmt.frequency + half_width_integral, alpha=0.2, color='red')
    
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
plt.xlabel('f (Hz)')
plt.title(' Fourier transforms of {} consecutive pulses, \ntimestamp : {}'.format(msmt.number_of_pulses, msmt.timestamp))
# plt.legend()
plt.show()

plt.figure(figsize=(10,5))
integral = msmt.integral_fft(msmt.frequency-half_width_integral, msmt.frequency+half_width_integral)
plt.plot(range(1,msmt.number_of_pulses+1-1), integral[:-1], 'o')
plt.xlabel('Pulse number')
plt.ylabel('Integral')
plt.title('Integral of the shaded area for each pulse {} , \ntimestamp : {}'.format(msmt.where, msmt.timestamp))
plt.show()
delta_ampl = (integral.max()-integral.min())/integral.max()
print "Lowest integral is {:.2f}% lower than highest".format(delta_ampl*100)
fidelity = 1 - np.pi**2/4*delta_ampl**2
print "This gives rise to an infidelity of {:.4f} (infidelity: {:.2f}%).".format(fidelity, (1-fidelity)*100)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(msmt.freq[0], msmt.mean_fft, 'k', label='Mean')
plt.fill_between(msmt.freq[0],  msmt.mean_fft - msmt.std_fft, msmt.mean_fft + msmt.std_fft, 
                 facecolor='blue', alpha=0.5, label=r'$\pm$ 1 SD')

print "                       {} {}".format(msmt.setup, msmt.where)
# set_limits()
plt.xlabel('f (Hz)')
plt.title('Mean Fourier transform of {} consecutive pulses , timestamp : {}'.format(msmt.number_of_pulses, msmt.timestamp))
plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)


if msmt.__class__.__name__ == 'enveloppe_msmt':
    env, t = hermite_pulse_env(env_amplitude=np.max(msmt.amplitude)*.91, mu=msmt.middle[pulse], T_herm=msmt.t_herm)
#     plt.plot(t, env, '--', label='Ideal')
#     plt.plot(msmt.awg_time-1.9865e-5, msmt.imod/1.05, label='Send to AWG') #Plot input for the AWG
elif msmt.__class__.__name__ == 'pulse_msmt':
    env, t = hermite_pulse(env_amplitude=np.max(msmt.amplitude)*.96, frequency=msmt.frequency, mu=msmt.middle[pulse], T_herm=msmt.t_herm)
fft, freq = fourier(env, t)
plt.plot(freq, fft, 'r--', label='Ideal')

plt.legend()
plt.show()

plt.figure(figsize=(6,3))
mean_fft_func = scipy.interpolate.interp1d(msmt.freq[0], msmt.mean_fft)
fft_func = scipy.interpolate.interp1d(freq, fft)
f = np.linspace(msmt.frequency - .3e8, msmt.frequency + .3e8, 10000)
# plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
ratio = mean_fft_func(f)/fft_func(f)
plt.plot(f, ratio)
plt.title("Ratio between mean FFT and ideal signal.")
plt.xlabel('f (Hz)')
plt.show()

# plt.figure(figsize=(10,3))
# plt.plot(freq[msmt][0],  std, label="Noise on RF signal")
# # plt.plot([1.66e-9, 1.8e-9], [noise_oscilloscope, noise_oscilloscope], 'k--', label="Noise oscilloscope")
# plt.xlim(msmt.frequency - 1e8, msmt.frequency + 1e8)
# plt.ylim(0,5e-10)
# plt.xlabel('f (Hz)')
# plt.title('Standard deviation of consequtive pulses, timestamp : {}'.format(timestamp[msmt]))
# plt.show()
# ind_low = int(np.argmin(np.abs(msmt.freq[0] - 1.705e9)))
# ind_high = int(np.argmin(np.abs(msmt.freq[0] - 1.735e9)))
# mean_plateau = np.mean(mean[ind_low:ind_high])
# print "The relative mean standard deviation to the peak is {:.3e}.".format(np.mean(std)/mean_plateau)

In [ ]:
# Exproting the complex fourier transform
t_step = (msmt.t[0][-1] - msmt.t[0][0])/msmt.t[0].size
fft = scipy.fftpack.fft(msmt.amplitude[0])
freq = scipy.fftpack.fftfreq(msmt.t[0].size, t_step)
np.savetxt('after_cryo_fft.csv', fft, delimiter=',')
np.savetxt('after_cryo_freq.csv', freq, delimiter=',')
plt.plot(freq, abs(fft))

# Analysis of multiple consecutive measurements

In [ ]:
plt.figure(figsize=(10,8))
# plt.plot(multi[measurement].freq[0], multi[measurement]mean)#, label='{}'.format(msmt))
for msmt in range(multi.number_of_msmts):
    plt.plot(multi.msmts[msmt].freq[0], multi.msmts[msmt].mean_fft, label='{}'.format(msmt))
plt.title('Mean Fourier transforms of {} measurements of {} consecutive pulses, \ntimestamps : {} - {}'
          .format(multi.number_of_msmts, multi.msmts[0].number_of_pulses, multi.msmts[multi.number_of_msmts-1].timestamp, multi.msmts[0].timestamp))
plt.xlabel('f (Hz)')
plt.xlim(multi.msmts[0].frequency - 1e8, multi.msmts[0].frequency + 1e8)
plt.show()

# Transfer function

In [ ]:
import csv
import scipy.interpolate

In [ ]:
class transfer_function(object):
    def __init__(self, filename): # Converted true if file is converted using the Inritsu software
        with open(filename) as csvfile:
            self.data = csv.reader(csvfile, delimiter=',')
            freq = []
            transmission = []
            for line_num, line in enumerate(self.data):
                if line_num >34: # Discard measurement settings
                    freq.append(line[1])
                    transmission.append(line[2])
            self.freq = np.array(freq, dtype=float)*1e6 # Make unit Hz
            self.transmission = np.array(transmission, dtype=float)
            self.norm_transmission = self.transmission-self.transmission.mean()
            self.lin_transmission = 10**(self.transmission/10.0)
            self.norm_lin_transmission = 10**(self.norm_transmission/10.0)
            self.lin_correction = 1 / self.norm_lin_transmission
            

setup = transfer_function('transfer_functions/whole_setup.csv')
amplifier = transfer_function('transfer_functions/amplifier.csv')
cryo = transfer_function('transfer_functions/cryo.csv')

In [ ]:
# # Linear plot
# plt.figure(figsize=(10, 6))
# plt.plot(setup.freq, setup.lin_transmission/setup.lin_transmission.mean(), label='Whole setup')
# plt.plot(cryo.freq, cryo.lin_transmission/cryo.lin_transmission.mean(), label='Cryostat and sample')
# plt.plot(amplifier.freq, amplifier.lin_transmission/amplifier.lin_transmission.mean(), label='Amplifier')
# plt.xlabel('f(Hz)')
# plt.ylabel('Transmission (a.u.)')
# plt.title('Transmission of different parts op the setup')
# plt.legend()
# plt.show()

# Logarithm plot
plt.figure(figsize=(10, 6))
plt.plot(setup.freq, setup.norm_transmission, label='Whole setup')
plt.plot(cryo.freq, cryo.norm_transmission, label='Cryostat and sample')
plt.plot(amplifier.freq, amplifier.norm_transmission, label='Amplifier')
plt.xlabel('f(Hz)')
plt.ylabel('Transmission (dB)')
plt.title('Transmission of different parts op the setup')
plt.legend()
plt.show()

# Check multiplying cryostat/sample and amplifier
plt.figure(figsize=(10, 6))
plt.plot(setup.transmission, label='Whole setup')
plt.plot(amplifier.transmission + cryo.transmission, '--', label='Combination')
plt.xlabel('f(Hz)')
plt.ylabel('Transmission (dB)')
plt.legend()
plt.show()

# Plot the correction
plt.figure(figsize=(10, 6))
func_lin_cor = scipy.interpolate.interp1d(setup.freq, setup.lin_correction)
func_msmt = scipy.interpolate.interp1d(msmt.freq[0], msmt.mean_fft)
f = np.linspace(msmt.frequency - 1e8, msmt.frequency + 1e8, 1000)
multiplication = func_lin_cor(f) * func_msmt(f)
plt.plot(f, multiplication*2, '--', label='Corrected signal')
plt.plot(f, func_msmt(f), label='Measurement')
plt.plot(f, func_lin_cor(f)/1e8, label='Correction function')
plt.xlabel('f(Hz)')
plt.title('Signal corrected with the transfer funciton.')
plt.legend()
plt.show()